In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    refresh_schema=False
    )

graph

In [5]:
# ! pip install accelerate

In [7]:
## Dataset Movie 
movie_query= """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

movie_query

"\nLOAD CSV WITH HEADERS FROM\n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') | \n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') | \n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') | \n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n"

In [ ]:
graph.query(movie_query)

In [8]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {POB: STRING, name: STRING, YOB: INTEGER}
Company {name: STRING}
entrepreneur {POB: STRING, name: STRING, YOB: INTEGER}
Country {name: STRING}
Person {name: STRING, born: INTEGER}
Movie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT}
User {name: STRING, city: STRING, userId: INTEGER, age: INTEGER}
Post {postId: INTEGER, content: STRING, timestamp: DATE_TIME}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:Acted_In]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)
(:User)-[:POSTED]->(:Post)
(:User)-[:LIKES]->(:User)
(:User)-[:FRIEND]->(:User)


In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    model_name="Gemma2-9b-It"
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x17cd277a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x17cf3b830>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True
)

chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x11a9ae0c0>, cypher_generation_chain=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}')
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x17cd277a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x17cf3b830>, model_name='Gemma2-9b-It', model_kwargs={}, groq

In [14]:
response = chain.invoke({"query":"Who was the director of the movie Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie{title:"Casino"})<-[:DIRECTED]-(p:Person) RETURN p.name

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director of the movie Casino?',
 'result': 'Martin Scorsese  \n'}

In [15]:
response = chain.invoke({"query":"Who acted in the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})-[:ACTED_IN]->(p:Person) RETURN p.name 

Full Context:
[]

> Finished chain.


{'query': 'Who acted in the movie Casino',
 'result': "I don't know the answer. \n"}

In [16]:
response = chain.invoke({"query":"get a list of drama movie"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre{name:"Drama"}) RETURN m.title
Full Context:
[{'m.title': 'Waiting to Exhale'}, {'m.title': 'American President, The'}, {'m.title': 'Nixon'}, {'m.title': 'Casino'}, {'m.title': 'Sense and Sensibility'}, {'m.title': 'Money Train'}, {'m.title': 'Copycat'}, {'m.title': 'Powder'}, {'m.title': 'Leaving Las Vegas'}, {'m.title': 'Othello'}]

> Finished chain.


{'query': 'get a list of drama movie',
 'result': "'Nixon', 'American President, The', 'Sense and Sensibility', 'Othello'  \n"}

In [17]:
response = chain.invoke({"query":"How many movies has Oliver Stone directed in?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name:"Oliver Stone"})-[:DIRECTED]->(m:Movie)
RETURN count(m)
Full Context:
[{'count(m)': 2}]

> Finished chain.


{'query': 'How many movies has Oliver Stone directed in?',
 'result': "I don't know the answer. \n"}

In [18]:
response = chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:Acted_In]-(p:Person) RETURN p.name 

Full Context:
[]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': "I don't know the answer. \n"}

In [19]:
response = chain.invoke({"query":"How many artists are there in Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})-[:ACTED_IN]->(p:Person)
RETURN count(distinct p)
Full Context:
[{'count(distinct p)': 0}]

> Finished chain.


{'query': 'How many artists are there in Casino?',
 'result': "I don't know the answer. \n"}

In [21]:
# In some casas, it is unable to create correct Cypher Query and so unable to retrieve the relevant information from GraphDB
# so we will implement some prompting strategies

In [27]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [28]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template(
    "User input:{question}\n Cypher query:{query}"
)

prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question, create a syntactically very accurate Cypher query",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question", "schema"]
)

prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}], example_prompt=PromptTemplate(input_varia

In [29]:
print(prompt.format(question="How many artists are there?", schema="neo4j"))

You are a Neo4j expert. Given an input question, create a syntactically very accurate Cypher query

User input:How many artists are there?
 Cypher query:MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)

User input:Which actors played in the movie Casino?
 Cypher query:MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name

User input:How many movies has Tom Hanks acted in?
 Cypher query:MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)

User input:List all the genres of the movie Schindler's List
 Cypher query:MATCH (m:Movie {title: 'Schindler\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name

User input:Which actors have worked in movies from both the comedy and action genres?
 Cypher query:MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name

User input: How many artists are there?
Cypher query: 


In [30]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    cypher_prompt=prompt,
    verbose=True,
    allow_dangerous_requests=True,
    # exclude_types=["Genre"]  # if do not want to search based on Genre
)

chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x11a9ae0c0>, cypher_generation_chain=FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_G

In [31]:
chain.invoke("Which actors played in the movie Casino?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person)
RETURN a.name

Full Context:
[{'a.name': 'Robert De Niro'}, {'a.name': 'Joe Pesci'}, {'a.name': 'Sharon Stone'}, {'a.name': 'James Woods'}]

> Finished chain.


{'query': 'Which actors played in the movie Casino?',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods played in the movie Casino.  \n'}

In [35]:
response = chain.invoke({"query":"How many artists are there in Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person)
RETURN count(DISTINCT a)

Full Context:
[{'count(DISTINCT a)': 4}]

> Finished chain.


{'query': 'How many artists are there in Casino?',
 'result': 'There are 4 artists in Casino. \n'}

In [36]:
response = chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person) 
RETURN a.name

Full Context:
[{'a.name': 'Robert De Niro'}, {'a.name': 'Joe Pesci'}, {'a.name': 'Sharon Stone'}, {'a.name': 'James Woods'}]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods were the actors of the movie Casino. \n'}